### Intro to Dataframes

In [ ]:
import pandas as pd

df = pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]], columns=['A','B','C'], index=['x','y','z'])

In [ ]:
df.describe()

### Loading in Dataframes from Files


In [ ]:
coffee = pd.read_csv("./warmup-data/coffee.csv")

results = pd.read_parquet("./data/results.parquet")

olympics = pd.read_excel("./data/olympics-data.xlsx")

bios = pd.read_csv("./data/bios.csv")


### Accessing data

In [ ]:
coffee.sample(10)

In [ ]:
coffee.loc[0:6,["Day","Units Sold"]]

In [ ]:
coffee.iloc[0:6,[0,2]] # Only using index

In [ ]:
coffee.loc[1:4,"Unit Solds"] = 10

coffee.head()

In [ ]:
coffee.at[0,"Unit Solds"]=100

In [ ]:
coffee.sort_values("Unit Solds",ascending=True)

coffee.head()

In [ ]:
for idx , row in coffee.iterrows():
    print(idx)
    print(row)
    print("\n")

### Filtering data

In [ ]:
bios.loc[bios['height_cm']>215,["name","height_cm"]]

In [ ]:
bios[(bios["height_cm"]>215)][["name","height_cm"]]

In [ ]:
bios[(bios["height_cm"]>215) & (bios["born_country"]=="USA")]

In [ ]:
bios[bios["name"].str.contains("Hristo",case=False)]

In [ ]:
# Top 5 tallest overall
bios.nlargest(5, 'height_cm')[['name', 'height_cm', 'born_country']]  # top 5 tallest rows


In [ ]:
# Tallest person in each country (per-group max)
bios[bios['height_cm'].eq(bios.groupby('born_country')['height_cm'].transform('max'))][['name', 'born_country', 'height_cm']]  # tallest per country

In [ ]:
# Filter by a set of countries
bios[bios['born_country'].isin(['USA', 'Germany', 'Bulgaria'])][['name', 'born_country', 'height_cm']]  # only selected countries


In [ ]:
# Exclude certain countries
bios[~bios['born_country'].isin({'USA', 'Canada'})][['name', 'born_country', 'height_cm']].head(10)  # not USA/Canada


In [ ]:
# Names that start with A or H (case-insensitive)
bios[bios['name'].str.contains(r'^(A|H)', case=False, na=False)][['name']].head(10)  # anchors at start


In [ ]:
# Duplicate names (possible homonyms)
bios[bios['name'].duplicated(keep=False)].sort_values('name')[['name', 'born_country', 'height_cm']]  # all duplicates


In [ ]:
# Data-quality check: names containing digits
bios[bios['name'].str.contains(r'\d', na=False)][['name']]

In [ ]:
# Shortest person in each country (per-group min via sort+drop_duplicates)
(bios.sort_values(['born_country', 'height_cm'])
     .drop_duplicates('born_country'))[['name', 'born_country', 'height_cm']]  # shortest per country


In [ ]:
bios.query('born_country== "USA" and height_cm > 200')[['name', 'born_country', 'height_cm']]

### Adding / Removing Columns

In [ ]:
coffee['price'] = 5

import numpy as np

coffee['new_price'] = np.where(coffee['Coffee Type'] == 'Espresso',3.99,4.99)

coffee.drop(columns=['price'], inplace=True)

coffee.head()

In [ ]:
## Pointing to the same memory space
coffe_new = coffee

## New one , not modifying the original
coffe_new = coffee.copy()

In [ ]:
coffee.head()

In [ ]:
coffee['revenue'] = coffee['Units Sold'] * coffee['new_price']

In [ ]:
coffee.rename(columns={'new_price':'price'}, inplace=True)

In [ ]:
bios.head()

In [ ]:
bios['first_name'] = bios['name'].str.split(' ').str[0]


bios['born_data_time']=pd.to_datetime(bios['born_date'])

In [ ]:
bios['height_category'] = bios['height_cm'].apply(lambda x : 'Short' if x < 165 else ('Average' if x<185 else 'Tall'))

In [ ]:
def categorize_athlete(row):
    if row['height_cm'] < 175 and row['weight_kg'] < 70:
        return 'Lightweight'
    elif row['height_cm'] < 185 and row['weight_kg'] <= 80:
        return 'Middleweight'
    else:
        return 'Heavyweight'

bios['category'] = bios.apply(categorize_athlete, axis=1)

### Merging & Concatenating Data

In [ ]:
nocs = pd.read_csv('./data/noc_regions.csv')

In [ ]:
nocs.head()

In [ ]:
bios = pd.merge(bios, nocs, left_on='born_country', right_on='NOC', how='left')

bios.rename(columns={'region':'born_country_full'},inplace=True)

In [ ]:
usa = bios[bios.born_country=='USA'].copy()
gbr = bios[bios.born_country=='GBR'].copy()

new_df = pd.concat([usa,gbr])

new_df.head()

### Handling Null Values

In [ ]:
coffee.loc[[0,1],'Units Sold'] = np.nan

In [ ]:
display(coffee)

In [ ]:
coffee= coffee.fillna(coffee['Units Sold'].mean())

In [ ]:
coffee.dropna(subset=['Units Sold'], inplace=True)

In [ ]:
coffee[coffee['Units Sold'].notna()]

### Aggregating Data

In [ ]:
bios.head()

In [ ]:
bios['born_city'].value_counts()

In [ ]:
bios[bios['born_country']=='USA']['born_region'].value_counts()

In [ ]:
coffee.groupby(['Coffee Type'])['Units Sold'].sum()

In [ ]:
coffee.groupby(['Coffee Type']).agg({'Units Sold':'sum','price':'mean'})

In [ ]:
pivot = coffee.pivot(columns='Coffee Type', index='Day' , values = 'revenue')

pivot

In [ ]:
bios.groupby(bios['born_date'])['name'].count().reset_index()

In [ ]:
# Total, average, min, max height per country
bios.groupby('born_country').agg({
    'height_cm': ['count', 'mean', 'min', 'max', 'std']
})


In [ ]:
bios.groupby('born_country').agg(
    num_people=('name', 'count'),
    avg_height=('height_cm', 'mean'),
    tallest=('height_cm', 'max'),
    shortest=('height_cm', 'min')
)


In [ ]:
# Top 10 countries with highest average height
bios.groupby('born_country')['height_cm'].mean().sort_values(ascending=False).head(10)


In [ ]:
# Ensure born_country is a column
if 'born_country' not in bios.columns:
    bios = bios.reset_index()

# Make sure height_cm is numeric
bios['height_cm'] = pd.to_numeric(bios['height_cm'], errors='coerce')

idx = bios.groupby('born_country')['height_cm'].idxmax()
idx = idx.dropna().astype(int)   # remove groups where max is NaN

out = bios.loc[idx, ['born_country', 'name', 'height_cm']].sort_values('born_country')
print(out)


### Advanced Funcs

In [ ]:
coffee['yesterday_revenue'] = coffee['revenue'].shift(2)

In [ ]:
coffee.head()

In [ ]:
bios.head()

In [ ]:
bios['height_rank'] = bios['height_cm'].rank()

bios.sort_values(['height_cm'],ascending=True)

In [ ]:
latte = coffee[coffee['Coffee Type']=='Latte'].copy()

latte['3day']=latte['Units Sold'].rolling(3).sum()
latte